# Pipeline description

In this experiment, we implemented a basic pipeline for RAG over structured data with below steps:
1. Pre-processed the tables and added metadata from the ‘context’ column
2. Encoded the texts from step 2 with SentenceTransformer('all-mpnet-base-v2')
3. Stored the encodings with the hnswlib package.
4. Retrieved the top 5 tables given the questions in queries.parquet
5. Implemented Llama 3 model (API) for answer generation

In [ ]:
cd /content/drive/MyDrive/2024_summer/RAG

/content/drive/MyDrive/2024_summer/RAG


In [ ]:
!pip install pandas pyarrow
!pip install hnswlib
!pip install hnswlib sentence-transformers
!pip install sacrebleu
!pip install replicate

  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
  Created wheel for hnswlib: filename=hnswlib-0.8.0-cp310-cp310-linux_x86_64.whl size=2331681 sha256=0df6b8ae036fe177dc37c43f1aa4d201550ca09c04a481a1a637fd95c05b8ac9
  Stored in directory: /root/.cache/pip/wheels/af/a9/3e/3e5d59ee41664eb31a4e6de67d1846f86d16d93c45f277c4e7
Successfully built hnswlib
  Using cached nvidia_cuda_nvrtc_cu12-12.1.105-py3-none-manylinux1_x86_64.whl.metadata (1.5 kB)
  Using cached nvidia_cuda_runtime_cu12-12.1.105-py3-none-manylinux1_x86_64.whl.metadata (1.5 kB)
  Using cached nvidia_cuda_cupti_cu12-12.1.105-py3-none-manylinux1_x86_64.whl.metadata (1.6 kB)
  Using cached nvidia_cudnn_cu12-8.9.2.26-py3-none-manylinux1_x86_64.whl.metadata (1.6 kB)
  Using cached nvidia_cublas_cu12-12.1.3.1-py3-none-manylinux1_x86_64.whl.metadata (1.5 kB)
  Using cached nvidia_cufft_cu12-11.0.2.54-py3-none-manylinux1_x86_64.whl.metadata (1.5 kB)


In [ ]:
import pandas as pd
import hnswlib
import sacrebleu
import numpy as np
from sentence_transformers import SentenceTransformer
import time
from sklearn.metrics import classification_report

/usr/local/lib/python3.10/dist-packages/sentence_transformers/cross_encoder/CrossEncoder.py:11: TqdmExperimentalWarning: Using `tqdm.autonotebook.tqdm` in notebook mode. Use `tqdm.tqdm` instead to force console mode (e.g. in jupyter console)
  from tqdm.autonotebook import tqdm, trange


## Load the files + Pre-preocessing

In [ ]:
# Read parquet files
queries = 'queries.parquet'
tables = 'corpus.parquet'

queries_df = pd.read_parquet(queries)
tables_df = pd.read_parquet(tables)

def pre_processing(arrays):
  result = []
  for array in arrays:
    new_array = "|".join(array)
    result.append(new_array)
  return result

tables_df['table'] = tables_df['table'].apply(pre_processing)
tables_df['table_context'] = tables_df.apply(lambda x: str(x['context'])+ ': ' +str(x['table']), axis = 1)

## 2. Retriever

In [ ]:
# references:
# https://github.com/nmslib/hnswlib
# https://esteininger.medium.com/building-a-vector-search-engine-using-hnsw-and-cosine-similarity-753fb5268839
# https://www.sbert.net/docs/sentence_transformer/pretrained_models.html

def fetch_k(tables,queries,encoder):

  #model = SentenceTransformer('all-mpnet-base-v2')# this model generates 768 dense vector space
  model = SentenceTransformer(encoder) #smaller encoder 384 dimension

  # Generate and store embeddings in HNSW
  # Create embeddings
  embeddings = model.encode(tables)

  # Get the dimension of our vector space
  dimension = embeddings.shape[1]
  print(dimension)
  # Create a new index
  p = hnswlib.Index(space = 'cosine', dim = dimension)
  # Initialize an index - the maximum number of elements should be known beforehand
  p.init_index(max_elements = 10000,ef_construction=200, M=16) # what are these?
  # max_elements: define the maximum number of elements that can be stored in the structure
  # ef_construction: define a construction/accuracy trade-off
  # M: defines the maximum number of outgoing connection to the graph

  # Element insertion (can be called several times)
  p.add_items(embeddings)
  # Controlling the recall by setting ef:
  p.set_ef(50) # ef should always be > k
  new_embedding = model.encode(queries)
  # Fetch k neighbours
  labels, distances = p.knn_query(new_embedding, k = 5)
  return labels, distances

# baseline approach
labels, distances = fetch_k(tables_df['table_context'], queries_df['query'],'all-MiniLM-L6-v2')

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:89: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


modules.json:   0%|          | 0.00/349 [00:00<?, ?B/s]

config_sentence_transformers.json:   0%|          | 0.00/116 [00:00<?, ?B/s]

README.md:   0%|          | 0.00/10.7k [00:00<?, ?B/s]

sentence_bert_config.json:   0%|          | 0.00/53.0 [00:00<?, ?B/s]

...

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/466k [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/112 [00:00<?, ?B/s]

1_Pooling/config.json:   0%|          | 0.00/190 [00:00<?, ?B/s]

384


### Classification report

In [ ]:
# considering only the top-1 retrieved table
y_fetch = [tops[0] for tops in labels]
y_gold = queries_df['database_id']
report = classification_report(y_gold, y_fetch)
print(report)
queries_df['retrieved_table'] = y_fetch

              precision    recall  f1-score   support

         0.0       0.00      0.00      0.00         1
         1.0       0.00      0.00      0.00         1
         2.0       1.00      1.00      1.00         1
         3.0       1.00      1.00      1.00         1
         4.0       0.00      0.00      0.00         1
         5.0       0.00      0.00      0.00         1
         6.0       0.00      0.00      0.00         1
         7.0       1.00      1.00      1.00         1
         8.0       0.00      0.00      0.00         1
         9.0       0.00      0.00      0.00         1
        10.0       1.00      1.00      1.00         1
        11.0       0.00      0.00      0.00         1
        12.0       0.00      0.00      0.00         1
        13.0       0.00      0.00      0.00         1
        14.0       0.00      0.00      0.00         1
        15.0       1.00      1.00      1.00         1
        16.0       0.00      0.00      0.00         1
        17.0       1.00    

/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1471: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1471: UndefinedMetricWarning: Recall and F-score are ill-defined and being set to 0.0 in labels with no true samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1471: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.

In [ ]:
# considering all top-k tables
y_fetch_5 = []
retrieved_tables = []
for gold, tops in zip(y_gold,labels):
  retrieved_tables.append(tops)
  if gold in tops:
    y_fetch_5.append(gold)
  else:
    y_fetch_5.append(tops[0])

report = classification_report(y_gold, y_fetch_5)
queries_df['retrieved_tables'] = retrieved_tables
print(report)

              precision    recall  f1-score   support

         0.0       1.00      1.00      1.00         1
         1.0       0.00      0.00      0.00         1
         2.0       1.00      1.00      1.00         1
         3.0       1.00      1.00      1.00         1
         4.0       0.00      0.00      0.00         1
         5.0       0.00      0.00      0.00         1
         6.0       0.00      0.00      0.00         1
         7.0       1.00      1.00      1.00         1
         8.0       0.00      0.00      0.00         1
         9.0       1.00      1.00      1.00         1
        10.0       1.00      1.00      1.00         1
        11.0       1.00      1.00      1.00         1
        12.0       1.00      1.00      1.00         1
        13.0       0.00      0.00      0.00         1
        14.0       0.00      0.00      0.00         1
        15.0       1.00      1.00      1.00         1
        16.0       1.00      1.00      1.00         1
        17.0       1.00    

/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1471: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1471: UndefinedMetricWarning: Recall and F-score are ill-defined and being set to 0.0 in labels with no true samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1471: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.

## LLaMa Answer Generation

In [ ]:
# get a token: https://replicate.com/account
# document: https://colab.research.google.com/drive/1K91q4p-OhL96FHBAVLsv9FlwFdu6Pn3c#scrollTo=5ZTkrAUhoIJE
from getpass import getpass
import os
REPLICATE_API_TOKEN = getpass()
os.environ["REPLICATE_API_TOKEN"] = REPLICATE_API_TOKEN

KeyboardInterrupt: Interrupted by user

In [ ]:
import replicate
import pandas as pd
import json


def call_llama(q,t):

    t = tables_df.iloc[int(t)]['table_context']
    system_prompt = "I will provide you with a query. Please answer the question by considering the table I provide."
    prompt = f"""The query is: {q} and the relevant table is: {t}"""


    input = {
    "top_k": 0,
    "top_p": 0.9,
    "min_tokens" : 0,
    "max_tokens":1024,
    "seed" : 240425,
    "temperature": 0.6,
        "system_prompt": system_prompt,
        "length_penalty": 1,
        "presence_penalty": 1.15,
        "log_performance_metrics": False,

        "prompt": prompt}

    iterator = replicate.run("meta/meta-llama-3.1-405b-instruct", input=input)
    # for event in replicate.stream(
    #     "meta/meta-llama-3-70b-instruct",
    #     input=input):
    #     print(event, end="")

    result = ''
    for w in iterator:
        result = result + w
    print(result)
    print('--;--')

    return result

test_queries_df = queries_df.head(10)
test_queries_df['llama_3_output'] = test_queries_df.apply(lambda row: call_llama(row['query'], row['retrieved_table']), axis = 1)
test_queries_df.to_pickle('result_top10.pkl')

## Result examination

In [ ]:
test_queries_df = pd.read_pickle('result_top10.pkl')
hypothesis = list(test_queries_df['llama_3_output'])
references = list(test_queries_df['answer'])
queries = test_queries_df['query']
bleu = sacrebleu.corpus_bleu(hypothesis, references)
#test_queries_df

In [ ]:
bleu

BLEU = 0.10 0.8/0.1/0.1/0.0 (BP = 1.000 ratio = 50.100 hyp_len = 501 ref_len = 10)

In [ ]:
index = 2
print("Database_id: ", index)
print()
print(f'Table {index}: ')
print(tables_df.iloc[index]['table_context'])
print()
print('Query: ',queries[index])
print()
print('Reference: ')
print(references[index])
print()
print()
print()
retrieved_table = int(test_queries_df['retrieved_table'].iloc[index])

if retrieved_table != index:
  print('Wrong table retrieved')
print("Retrieved_table ID: ")
print(retrieved_table)
print()
print('Retrieved tables:')
print(tables_df.iloc[retrieved_table]['table_context'])
print()
print('LLaMA 3 output: ')
print(hypothesis[index])
print()
print('Top-five tables')
print(test_queries_df['retrieved_tables'].iloc[index])

Database_id:  2

Table 2: 
{'table_page_title': 'Dyro', 'table_section_title': 'Awards and nominations'}: ['Year|Award|Nominee|Category|Result', '2013|DJ Magazine Awards|Dyro|Top 100 DJs|30', '2014|DJ Magazine Awards|Dyro|Top 100 DJs|27', '2015|DJ Magazine Awards|Dyro|Top 100 DJs|27', '2016|DJ Magazine Awards|Dyro|Top 100 DJs|93']

Query:  Dyro ranked how high and in what category for what award in 2014?

Reference: 
In 2014, Dyro ranked No. 27 on the DJ Mag Top 100 DJs list.



Retrieved_table ID: 
2

Retrieved tables:
{'table_page_title': 'Dyro', 'table_section_title': 'Awards and nominations'}: ['Year|Award|Nominee|Category|Result', '2013|DJ Magazine Awards|Dyro|Top 100 DJs|30', '2014|DJ Magazine Awards|Dyro|Top 100 DJs|27', '2015|DJ Magazine Awards|Dyro|Top 100 DJs|27', '2016|DJ Magazine Awards|Dyro|Top 100 DJs|93']

LLaMA 3 output: 
Dyro ranked 27th in the Top 100 DJs category for the DJ Magazine Awards in 2014.

Top-five tables


KeyError: 'retrieved_tables'

In [ ]:
bleu.score

0.10290789719064093